# ANN with MLflow

In [3]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [ ]:
# Load the data of wine quality dataset
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=';')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
# Split the dataset into training, testing and validation sets
train, test = train_test_split(data, test_size=0.25, random_state=42)

# Independent and dependent variables
X_train = train.drop('quality', axis=1).values
y_train = train[['quality']].values.ravel()

## Validation set
X_test = test.drop('quality', axis=1).values
y_test = test['quality'].values.ravel()

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

signature = infer_signature(X_train, y_train)

In [6]:
def train_model(params, epochs, X_train, y_train, X_val, y_val, X_test, y_test):
    # Define the model architecture
    mean=np.mean(X_train,axis=0) # Required for Normalization layer
    var=np.var(X_train,axis=0)

    model=keras.Sequential(
        [
            keras.Input([X_train.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"],momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(X_train,y_train,validation_data=(X_val,y_val),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(X_val,y_val,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [7]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs = 3,
        X_train = X_train,
        y_train = y_train,
        X_val = X_val,
        y_val = y_val,
        X_test = X_test,
        y_test = y_test,
    )
    return result

In [ ]:
# Define the hyperparameter search space
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 0.9),
}

In [9]:
# Set up MLflow experiment
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature = signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2025/08/07 20:01:28 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.


E0000 00:00:1754618488.898395   23082 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1754618488.900682   23082 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 22s 490ms/step - loss: 35.5591 - root_mean_squared_error: 5.9631
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 3.0674 - root_mean_squared_error: 1.7514 - val_loss: 0.8609 - val_root_mean_squared_error: 0.9279

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.8586 - root_mean_squared_error: 0.9266
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6927 - root_mean_squared_error: 0.8323 - val_loss: 0.6366 - val_root_mean_squared_error: 0.7979

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.5317 - root_mean_squared_error: 0.7292
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5823 - root_mean_squared_error: 0.7631 - val_loss: 0.5656 - val_root_mean_squared_error: 0.7521

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.5276 - root_mean_squared_error: 0.7264
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5656 - root_mean_squared_error: 0

2025/08/07 20:01:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.

2025/08/07 20:01:35 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



🏃 View run efficient-wolf-891 at: http://127.0.0.1:5000/#/experiments/416204377740878185/runs/eeebedf00a7540e1ada95d2440fc7273

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/416204377740878185

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 10s 238ms/step - loss: 30.5096 - root_mean_squared_error: 5.5235
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 31.7698 - root_mean_squared_error: 5.6365 - val_loss: 31.4875 - val_root_mean_squared_error: 5.6114

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 34.5066 - root_mean_squared_error: 5.8742
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 31.1745 - root_mean_squared_error: 5.5834 - val_loss: 30.8970 - val_root_mean_squared_error: 5.5585

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 30.7620 - root_mean_squar

...

2025/08/07 20:01:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.

2025/08/07 20:01:47 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



🏃 View run unique-worm-189 at: http://127.0.0.1:5000/#/experiments/416204377740878185/runs/23b46d5464b84705878a34db62bec331

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/416204377740878185  

100%|██████████| 4/4 [00:18<00:00,  4.71s/trial, best loss: 0.7520656585693359]

2025/08/07 20:01:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


2025/08/07 20:01:51 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Best parameters: {'lr': np.float64(0.01821963456823003), 'momentum': np.float64(0.6736624922858568)}
Best eval rmse: 0.7520656585693359
🏃 View run abundant-robin-630 at: http://127.0.0.1:5000/#/experiments/416204377740878185/runs/cc0bb69ca4bb4bf3acc9809d9f23d94e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/416204377740878185


In [10]:
# Load model as a PyFuncModel.
model_uri = 'runs:/cc0bb69ca4bb4bf3acc9809d9f23d94e/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(X_test))

/home/paola/personal/mlops-fundamentals/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[6.3305354],
       [6.7223215],
       [6.264503 ],
       ...,
       [6.2506523],
       [6.734172 ],
       [5.6355553]], dtype=float32)

In [11]:
## Register in the model registry
mlflow.register_model(model_uri,"wine-quality")

Successfully registered model 'wine-quality'.
2025/08/07 20:12:09 WARNING mlflow.tracking._model_registry.fluent: Run with id cc0bb69ca4bb4bf3acc9809d9f23d94e has no artifacts at artifact path 'model', registering model based on models:/m-d7b1baab76b3441b8607dc9871fa3490 instead
2025/08/07 20:12:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: wine-quality, version 1
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1754619129120, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1754619129120, metrics=None, model_id=None, name='wine-quality', params=None, run_id='cc0bb69ca4bb4bf3acc9809d9f23d94e', run_link='', source='models:/m-d7b1baab76b3441b8607dc9871fa3490', status='READY', status_message=None, tags={}, user_id='', version='1'>